In [5]:
import pandas as pd

tablice= pd.read_csv('lists.csv', index_col='rank', encoding = "utf-8")

pd.options.display.max_rows = 20

najprej poglejmo, kako sploh izgledajo tablice 



In [7]:
tablice

,date,time,name,position,DOB,venue,nationality
rank,,,,,,,
1,31 AUG 2001,14:29.32,Olga YEGOROVA,1,28 MAR 1972,Berlin (GER),RUS
2,31 AUG 2001,14:31.69,Gete WAMI,2,11 DEC 1974,Berlin (GER),ETH
3,31 AUG 2001,14:32.44,Paula RADCLIFFE,3,17 DEC 1973,Berlin (GER),GBR
4,31 AUG 2001,14:39.22,Tatyana TOMASHOVA,4,01 JUL 1975,Berlin (GER),RUS
5,24 JUN 2001,14:40.47,Yelena ZADOROZHNAYA,1,03 DEC 1977,Bremen (GER),RUS
6,17 JUL 2001,14:45.86,Edith MASAI,1,04 APR 1967,Stockholm (SWE),KEN
7,13 JUL 2001,14:46.92,Gabriela SZABO,2,14 NOV 1975,Oslo (NOR),ROU
8,13 JUL 2001,14:48.97,Rose CHERUIYOT,3,21 JUL 1976,Oslo (NOR),KEN
9,13 JUL 2001,14:49.32,Asmae LEGHZAOUI,4,30 AUG 1976,Oslo (NOR),MAR


In [9]:
tablice.loc[100]


,date,time,name,position,DOB,venue,nationality
rank,,,,,,,
100,08 JUL 2001,15:34.64,Yumiko OKAMOTO,2,28 MAR 1978,Sapporo (JPN),JPN
100,15 DEC 2002,15:37.4h,Risa HAGIWARA,1,22 NOV 1982,Yokohama (JPN),JPN
100,27 SEP 2003,15:34.40,Miwako YAMANAKA,5,24 MAY 1978,Okayama (JPN),JPN
100,06 JUN 2004,15:27.43,Megumi TANAKA-OSHIMA,7,04 SEP 1975,Tottori (JPN),JPN
100,04 JUN 2005,15:29.96,Yoshiko FUJINAGA,6,15 AUG 1981,Tokyo (JPN),JPN
100,15 AUG 2006,15:32.34,Florence KIPLAGAT,2,27 FEB 1987,Beijing (CHN),KEN
100,29 APR 2007,15:34.35,Hitomi NIIYA,6,26 FEB 1988,Hiroshima (JPN),JPN
100,28 JUN 2008,15:34.07,Elena ROMAGNOLO,2,05 OCT 1982,Firenze (ITA),ITA
100,26 JUN 2009,15:38.61,Nicole BLOOD,7,25 APR 1988,Eugene (USA),USA


In [11]:
tablice.sort_values('time').head(50)

,date,time,name,position,DOB,venue,nationality
rank,,,,,,,
1,06 JUN 2008,14:11.15,Tirunesh DIBABA,1,01 JUN 1985,Oslo (NOR),ETH
1,02 JUN 2016,14:12.59,Almaz AYANA,1,21 NOV 1991,Roma (ITA),ETH
2,22 JUL 2008,14:12.88,Meseret DEFAR,1,19 NOV 1983,Stockholm (SWE),ETH
1,17 MAY 2015,14:14.32,Almaz AYANA,1,21 NOV 1991,Shanghai (CHN),ETH
2,04 JUL 2015,14:15.41,Genzebe DIBABA,1,08 FEB 1991,Paris (FRA),ETH
1,15 JUN 2007,14:16.63,Meseret DEFAR,1,19 NOV 1983,Oslo (NOR),ETH
1,08 JUN 2017,14:18.37,Hellen OBIRI,1,13 DEC 1989,Roma (ITA),KEN
1,29 JUL 2011,14:20.87,Vivian Jepkemoi CHERUIYOT,1,11 SEP 1983,Stockholm (SWE),KEN
2,15 JUN 2007,14:22.51,Vivian Jepkemoi CHERUIYOT,2,11 SEP 1983,Oslo (NOR),KEN
